Import Necessary Packages

In [2]:
import pandas as pd
import psycopg2

Simple Diagram showing the simple star schema we will be building

<img src='../images/star_schema.png' alt="Alternative text" />

In [3]:
df = pd.read_csv("../data/clean_college_data.csv")
df.head()

,Unnamed: 0,date,state,county,city,ipeds_id,college,cases,cases_2021
0,0,2021-05-26,Alabama,Madison,Huntsville,100654,Alabama A&M University,41,0.0
1,1,2021-05-26,Alabama,Montgomery,Montgomery,100724,Alabama State University,2,0.0
2,2,2021-05-26,Alabama,Limestone,Athens,100812,Athens State University,45,10.0
3,3,2021-05-26,Alabama,Lee,Auburn,100858,Auburn University,2742,567.0
4,4,2021-05-26,Alabama,Montgomery,Montgomery,100830,Auburn University at Montgomery,220,80.0


Determining columns for the fact and dimention table as per the schema

In [4]:
dim_cols = ['ipeds_id', 'college', 'county', 'state']
fact_cols = ['ipeds_id', 'cases', 'cases_2021']

In [5]:
df_dim = df[dim_cols]
df_dim.head()

,ipeds_id,college,county,state
0,100654,Alabama A&M University,Madison,Alabama
1,100724,Alabama State University,Montgomery,Alabama
2,100812,Athens State University,Limestone,Alabama
3,100858,Auburn University,Lee,Alabama
4,100830,Auburn University at Montgomery,Montgomery,Alabama


In [43]:
df_fact = df[fact_cols]
type(df_fact['cases'].to_list()[108])

int

SQL Commands

In [48]:
dim_create = '''
CREATE TABLE IF NOT EXISTS Dim_college_info(
    ipeds_id varchar,
    college varchar,
    county varchar,
    state  varchar
);'''

fact_create = ''' 
CREATE TABLE IF NOT EXISTS fact_covid_cases(
    case_id serial,
    ipeds_id varchar,
    cases integer,
    cases_2021 float
);'''


dim_insert = ''' 
    INSERT INTO Dim_college_info(
        ipeds_id, 
        college, 
        county,
        state)

    VALUES (
        %s, %s, %s, %s);
    '''

fact_insert = ''' 
    INSERT INTO fact_covid_cases(
    ipeds_id,
    cases,
    cases_2021)

    VALUES (
        %s, %s, %s);
    '''

Connect To database and create cursor

In [9]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 port=5433 dbname=covid_warehouse user=postgres password=testing321")
    print("Connected Successfully")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

Connected Successfully


In [11]:
try:
    cur = conn.cursor()
    print("Cursor Created")
except psycopg2.Error as e:
    print("NO Cursor created")
    print(e)

Cursor Created


Execute the queries

In [44]:
try:
    cur.execute(dim_create)
    cur.execute(fact_create)
except psycopg2.Error as e:
    print("Error while creating table")
    print(e)

In [45]:
conn.commit()
print("Successfully created tables")

Successfully created tables


In [52]:
## Function to Insert data from Dataframe

def insert_from_df(data_frame, sql_command):
    try: 
        for i, row in data_frame.iterrows():
            cur.execute(sql_command, list(row))
        print("Successfully Inserted data")
    except psycopg2.Error as e:
        print("Could Not Insert Data")
        print(e)

        

Insert Data